In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets
import matplotlib.pyplot as plt
%matplotlib inline

### hyperparameter tunning

In [2]:
num_epochs = 50
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32,32,3)
num_classes = 10

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32,(3,3), padding = 'SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32,(3,3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64,(3,3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64,(3,3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs= inputs, outputs=net, name = 'Basic_CNN')

In [4]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
            loss = 'sparse_categorical_crossentropy',
            metrics = ['accuracy'])

### data preprocess

In [5]:
train_paths = glob('cifar/train/*.png')[:1000]
test_paths = glob('cifar/test/*.png')[:1000]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png','')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png','')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot) # label번호를 반환

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls = AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls = AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size = len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls = AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

### train

In [14]:
steps_per_epoch = len(train_paths) //batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch = steps_per_epoch,
    validation_data = test_dataset,
    validation_steps = validation_steps,
    epochs = num_epochs,)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
31/31 [==============================] - 4s 123ms/step - loss: 2.3117 - accuracy: 0.1157 - val_loss: 2.2880 - val_accuracy: 0.0958
Epoch 2/50
31/31 [==============================] - 4s 121ms/step - loss: 2.2710 - accuracy: 0.1322 - val_loss: 2.2906 - val_accuracy: 0.1159
Epoch 3/50
31/31 [==============================] - 4s 120ms/step - loss: 2.2240 - accuracy: 0.1591 - val_loss: 2.1703 - val_accuracy: 0.1885
Epoch 4/50
31/31 [==============================] - 4s 118ms/step - loss: 2.1911 - accuracy: 0.1715 - val_loss: 2.1566 - val_accuracy: 0.1744
Epoch 5/50
31/31 [==============================] - 4s 121ms/step - loss: 2.1202 - accuracy: 0.2389 - val_loss: 2.0738 - val_accuracy: 0.2339
Epoch 6/50
31/31 [==============================] - 4s 116ms/step - loss: 2.1121 - accuracy: 0.2108 - val_loss: 2.0850 - val_accuracy: 0.2288
Epoch 7/50
31/31 [==============================] - 4s 117ms/step - loss

### save model

In [19]:
save_path = 'mymodel.h5'

In [20]:
model.save(save_path, include_optimizer = True)

In [21]:
model = tf.keras.models.load_model('mymodel.h5')

### save model 2

In [22]:
model.save_weights('model_weights.h5')

In [23]:
with open('model_architecture.json','w') as f:
    f.write(model.to_json())

In [24]:
from tensorflow.keras.models import model_from_json

In [28]:
with open('model_architecture.json','r') as f:
    model = model_from_json(f.read())

In [29]:
model.load_weights('model_weights.h5')

### h5모델 들여다보기

In [30]:
import h5py

In [35]:
model_file = h5py.File('mymodel.h5','r+')

In [46]:
model_file.shape

AttributeError: 'File' object has no attribute 'shape'

In [36]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [38]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [42]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [43]:
weight = np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

In [44]:
weight.shape

(3, 3, 3, 32)